# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import cartopy.crs as ccrs
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,college,64.8569,-147.8028,25.18,51,75,1.54,US,1723068440
1,1,talnakh,69.4865,88.3972,9.11,74,0,3.24,RU,1723068441
2,2,stanley,54.8680,-1.6985,14.29,81,20,2.57,GB,1723068442
3,3,dunedin,-45.8742,170.5036,9.33,81,100,0.89,NZ,1723068443
4,4,tisul',55.7611,88.3122,15.34,96,57,3.09,RU,1723068444


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.0,
    color = "City"
)

# Display the map
map_plot_humidity

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
wind_data_df = city_data_df.loc[(city_data_df['Wind Speed']<= 5) ]
temp_data_df = wind_data_df.loc[(wind_data_df['Max Temp']>= 15) & (wind_data_df['Max Temp']<= 30) ]
vacation_data_df = temp_data_df.loc[(temp_data_df['Cloudiness']<= 50)]

# Drop any rows with null values
vacation_data_df.dropna()

# Display sample data
vacation_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
10,10,kurchum,48.5722,83.6542,19.16,61,38,2.83,KZ,1723068451
16,16,adamstown,-25.0660,-130.1015,18.98,65,26,3.52,PN,1723068458
17,17,uberlandia,-18.9186,-48.2772,24.56,33,0,4.63,BR,1723068190
20,20,ribeira grande,38.5167,-28.7000,22.65,82,8,2.68,PT,1723068462
32,32,sala,34.0389,-6.8166,23.13,94,20,1.54,MA,1723068477
...,...,...,...,...,...,...,...,...,...,...
560,560,messini,37.0511,22.0078,28.99,73,0,1.03,GR,1723069115
562,562,antalaha,-14.9003,50.2788,21.19,64,47,2.57,MG,1723069117
566,566,markaz-e woluswali-ye achin,34.1258,70.7078,27.68,70,34,0.87,AF,1723069121
571,571,ca mau,9.1769,105.1500,25.17,89,32,1.34,VN,1723069127


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = vacation_data_df.loc[: , ('City', 'Country', 'Lat', 'Lng', 'Humidity')].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,kurchum,KZ,48.5722,83.6542,61,
16,adamstown,PN,-25.0660,-130.1015,65,
17,uberlandia,BR,-18.9186,-48.2772,33,
20,ribeira grande,PT,38.5167,-28.7000,82,
32,sala,MA,34.0389,-6.8166,94,
...,...,...,...,...,...,...
560,messini,GR,37.0511,22.0078,73,
562,antalaha,MG,-14.9003,50.2788,64,
566,markaz-e woluswali-ye achin,AF,34.1258,70.7078,70,
571,ca mau,VN,9.1769,105.1500,89,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
limit = 10
radius = 10000
categories = "accommodation.hotel"

params = {
    "categories":categories,
    "filter":"",
    "limit":limit,
    "bias":"",
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")



Starting hotel search
kurchum - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
uberlandia - nearest hotel: Hotel Presidente
ribeira grande - nearest hotel: Hospedaria JSF
sala - nearest hotel: Le dawliz
port alfred - nearest hotel: No hotel found
taltal - nearest hotel: Hostería Taltal
horta - nearest hotel: Azoris Faial Garden - Resort Hotel
albany - nearest hotel: No hotel found
saint-joseph - nearest hotel: PALM Hotel & Spa
hawaiian paradise park - nearest hotel: No hotel found
santa elena de uairen - nearest hotel: Posada Villa Fairmont
sitka - nearest hotel: Aspen Suites Hotel
praia da vitoria - nearest hotel: Salles
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
georgetown - nearest hotel: Page 63 hostel
santa cruz - nearest hotel: Aviador Hotel Boutique
holualoa - nearest hotel: Kona Hotel
oltinko'l - nearest hotel: No hotel found
carnarvon - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
negotin - neares

In [7]:
# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
10,kurchum,KZ,48.5722,83.6542,61,No hotel found
16,adamstown,PN,-25.0660,-130.1015,65,No hotel found
17,uberlandia,BR,-18.9186,-48.2772,33,Hotel Presidente
20,ribeira grande,PT,38.5167,-28.7000,82,Hospedaria JSF
32,sala,MA,34.0389,-6.8166,94,Le dawliz
...,...,...,...,...,...,...
560,messini,GR,37.0511,22.0078,73,Kleopatra Inn
562,antalaha,MG,-14.9003,50.2788,64,Hôtel Florida
566,markaz-e woluswali-ye achin,AF,34.1258,70.7078,70,No hotel found
571,ca mau,VN,9.1769,105.1500,89,Khách sạn Song Ngọc


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot_hotel = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = 10,
    scale = 1.0,
    color = "City",
    hover_cols = (['Lng','Lat','City', 'Country','Hotel Name'])
   )

# Display the map
map_plot_hotel

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)